Калибровка за год до oot - 2016

In [36]:
import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as ss
from sklearn.linear_model import LinearRegression

In [2]:
samples_catboost = {
    k: pd.read_parquet(f"/home/asciishell/s3/jupyter.asciishell.ru/data_{k}.parquet") for k in ["train", "oos", "oot"]
}
samples_woe = {
    k: pd.read_parquet(f"/home/asciishell/s3/jupyter.asciishell.ru/data_{k}_with_score_WoE.parquet")
    for k in ["train", "oos", "oot"]
}

In [7]:
features = [
    # "inq_fi",
    # "all_util",
    "home_ownership",
    "log_annual_inc",
    "annual_inc",
    "num_rev_tl_bal_gt_0",
    "mort_acc",
    "funded_amnt_inv",
    "emp_title",
    "bc_open_to_buy",
    "debt_load",
    "fico_range_low",
    "fico_range_high",
]

cat_features = [
    "emp_title",
    "home_ownership",
    "verification_status",
    "purpose",
    "zip_code",
    "addr_state",
    "initial_list_status",
    "application_type",
    "verification_status_joint",
    "disbursement_method",
]
cat_features = sorted(set(cat_features) & set(features))
params = {
    "iterations": 400,
    "depth": 6,
    "l2_leaf_reg": 24.337217538245532,
    "learning_rate": 0.14119449759028455,
    "random_strength": 1.284731588293853,
    "rsm": 0.5363370106196876,
    "grow_policy": "SymmetricTree",
    "od_wait": 50,
    "eval_metric": "AUC",
    "logging_level": "Silent",
}

In [8]:
clf = catboost.CatBoostClassifier(**params)
clf.load_model("catboost.cb");

In [9]:
dfs_catboost = {k: v[v["target"] != -1] for k, v in samples_catboost.items()}  # [features + ["target"]]
dfs_woe = {k: v[v["target"] != -1] for k, v in samples_woe.items()}

pools = {
    k: catboost.Pool(v[v["target"] != -1][features], v[v["target"] != -1]["target"], cat_features=cat_features)
    for k, v in samples_catboost.items()
}

for key in pools:
    dfs_catboost[key]["score"] = clf.predict_proba(pools[key])[:, 1]

/tmp/ipykernel_11766/3105231172.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_catboost[key]["score"] = clf.predict_proba(pools[key])[:, 1]
/tmp/ipykernel_11766/3105231172.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_catboost[key]["score"] = clf.predict_proba(pools[key])[:, 1]
/tmp/ipykernel_11766/3105231172.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [92]:
df = dfs_woe["oos"]
df = df[(df["issue_d"].dt.year == 2016) & (df["issue_d"].dt.month > 6)]
lr = LinearRegression()
lr.fit(df["score"].values.reshape(-1, 1), df["target"])
lr.coef_, lr.intercept_

(array([1.29130818]), -0.0010174000394091443)

In [93]:
lr.predict(df["score"].values.reshape(-1, 1)).mean(), df["target"].mean()

(0.25479634616986424, 0.25479634616986424)

In [94]:
calibration_res = []
v = dfs_woe["oot"]
s = v[v["target"] != -1]
y_true = s["target"].values
y_score = lr.predict(s["score"].values.reshape(-1, 1))

lb, ub = [x / len(s) for x in ss.binom.interval(n=len(s), p=y_score.mean(), alpha=0.95)]
calibration_res.append(
    {
        "sample": "oot",
        "TR": y_true.mean(),
        "SR": y_score.mean(),
        "LB": lb,
        "UB": ub,
        "OK": lb <= y_true.mean() <= ub,
    }
)
calibration_res = pd.DataFrame(calibration_res).round(4)
calibration_res["sample"] = [
    x + (" 🟢" if y else " 🔴") for x, y in zip(calibration_res["sample"].values, calibration_res["OK"].values)
]
calibration_res

,sample,TR,SR,LB,UB,OK
0,oot 🟢,0.2365,0.2349,0.2331,0.2366,True


In [86]:
df = dfs_catboost["oos"]
df = df[(df["issue_d"].dt.year == 2016) & (df["issue_d"].dt.month > 2)]
lr = LinearRegression()
lr.fit(df["score"].values.reshape(-1, 1), df["target"])
lr.coef_, lr.intercept_

(array([1.17575344]), 0.02065090102185385)

In [87]:
lr.predict(df["score"].values.reshape(-1, 1)).mean(), df["target"].mean()

(0.2479012062691614, 0.24790120626916134)

In [88]:
calibration_res = []
v = dfs_catboost["oot"]
s = v[v["target"] != -1]
y_true = s["target"].values
y_score = lr.predict(s["score"].values.reshape(-1, 1))

lb, ub = [x / len(s) for x in ss.binom.interval(n=len(s), p=y_score.mean(), alpha=0.95)]
calibration_res.append(
    {
        "sample": "oot",
        "TR": y_true.mean(),
        "SR": y_score.mean(),
        "LB": lb,
        "UB": ub,
        "OK": lb <= y_true.mean() <= ub,
    }
)
calibration_res = pd.DataFrame(calibration_res).round(4)
calibration_res["sample"] = [
    x + (" 🟢" if y else " 🔴") for x, y in zip(calibration_res["sample"].values, calibration_res["OK"].values)
]
calibration_res

,sample,TR,SR,LB,UB,OK
0,oot 🟢,0.2365,0.2382,0.2364,0.2399,True
